In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 12 ms (started: 2021-08-12 14:14:21 +08:00)


In [4]:
%%bash

# 增加更新
git add *.ipynb

git remote -v

git commit -m '更新 ch1 #1 change Aug 12, 2021'

git push origin master

origin	git@github.com:ustchope/Advanced-Deep-Learning-with-Keras.git (fetch)
origin	git@github.com:ustchope/Advanced-Deep-Learning-with-Keras.git (push)
[master 79453f3] 更新 ch1 #1 change Aug 11, 2021
 1 file changed, 159 insertions(+), 133 deletions(-)


To git@github.com:ustchope/Advanced-Deep-Learning-with-Keras.git
   bd772a5..79453f3  master -> master


time: 4.3 s (started: 2021-08-11 15:00:26 +08:00)


In [ ]:
#设置使用的gpu
import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU")

# 自编码器

在上一章，第 2 章，深度神经网络中，我们介绍了深度神经网络的概念。 我们现在将继续研究自动编码器，它是一种神经网络架构，试图找到给定输入数据的压缩表示。

与前几章类似，输入数据可能有多种形式，包括语音、文本、图像或视频。 自编码器将尝试找到一个表示或一段代码，以便对输入数据执行有用的转换。 例如，在对自动编码器进行去噪时，神经网络将尝试找到可用于将嘈杂数据转换为干净数据的代码。 嘈杂的数据可以是带有静态噪声的录音形式，然后将其转换为清晰的声音。 自编码器将仅从数据中自动学习代码，无需人工标记。 因此，自动编码器可以归类为无监督学习算法。

在本书后面的章节中，我们将介绍生成对抗网络 (GAN) 和变分自动编码器 (VAE)，它们也是无监督学习算法的代表形式。 这与我们在前几章讨论的监督学习算法形成对比，后者需要人工注释。

总之，本章介绍：
* 自编码器的原理
* 如何使用 tf.keras 实现自动编码器
* 去噪和着色自编码器的实际应用

让我们首先了解什么是自动编码器，以及自动编码器的原理。

## 自编码器的原理

在最简单的形式中，自动编码器将通过尝试将输入复制到输出来学习表示或代码。 然而，使用自动编码器并不像将输入复制到输出那么简单。 否则，神经网络将无法发现输入分布中的隐藏结构。

自编码器将输入分布编码为低维张量，通常采用向量的形式。 这将逼近通常称为潜在表示、代码或向量的隐藏结构。 这个过程构成了编码部分。 然后潜在向量将由解码器部分解码以恢复原始输入。

由于潜在向量是输入分布的低维压缩表示，因此应该预期解码器恢复的输出只能近似于输入。 输入和输出之间的差异可以通过损失函数来衡量。

但是我们为什么要使用自编码器呢？ 简而言之，自动编码器以其原始形式或作为更复杂神经网络的一部分而具有实际应用。

它们是理解深度学习高级主题的关键工具，因为它们为我们提供了适合密度估计的低维数据表示。 此外，它可以被有效地处理以对输入数据执行结构化操作。 常见的操作包括去噪、着色、特征级算法、检测、跟踪和分割，仅举几例。

在本节中，我们将介绍自编码器的原理。 我们将使用前面章节中介绍的 MNIST 数据集来研究自动编码器。

首先，我们需要知道一个自动编码器有两个操作符，它们是：
* **编码器**：这将输入 x 转换为低维潜在向量，$𝒛 = 𝑓(𝒙)$。 由于潜在向量是低维的，编码器被迫只学习输入数据中最重要的特征。 例如，在 MNIST 数字的情况下，要学习的重要特征可能包括书写风格、倾斜角度、笔画的圆度、粗细等。 从本质上讲，这些是表示数字 0 到 9 所需的最重要的信息位。
* **解码器**：这试图从潜在向量中恢复输入，$𝑔(𝒛) = \tilde{x}$。

虽然潜在向量的维度很低，但它有足够的大小来允许解码器恢复输入数据。

解码器的目标是使 $\tilde{x}$ 尽可能接近 $x$。 通常，编码器和解码器都是非线性函数。 z 的维度是它可以表示的显着特征数量的度量。 该维度通常比输入维度小得多，以提高效率，并且为了限制潜在代码仅学习输入分布的最显着属性。

当潜在代码的维度明显大于 x 时，自动编码器倾向于记住输入。

一个合适的损失函数 $ℒ(𝒙, \tilde{x})$ 是衡量输入 x 与输出的不同程度的度量，输出是恢复的输入 $\tilde{x}$。 如以下等式所示，均方误差 (MSE) 是此类损失函数的一个示例：

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtcvgdo95ej61a2068dg302.jpg)

在这个例子中，m 是输出维度（例如，在 MNIST 中，m = 宽度 × 高度 × 通道 = 28 × 28 × 1 = 784）。 $x_i$ 和 $\tilde{x}_i$ 分别是 $x$ 和 $\tilde{x}$ 的元素。 由于损失函数是输入和输出之间差异的度量，我们能够使用替代的重建损失函数，例如二元交叉熵或结构相似性指数 (SSIM)。

与其他神经网络类似，自编码器会在训练期间尝试使此误差或损失函数尽可能小。 图 3.1.1 显示了一个自动编码器。 编码器是一个将输入 $x$ 压缩为低维潜在向量 $z$ 的函数。 这个潜在向量代表了输入分布的重要特征。 然后，解码器尝试以 $\tilde{x}$ 的形式从潜在向量中恢复原始输入。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtcvkbe28sj61d809mwf902.jpg)

为了将自动编码器放入上下文中，x 可以是一个 MNIST 数字，其维度为 28 × 28 × 1 = 784。编码器将输入转换为一个低维 z，它可以是一个 16 维的潜在向量。 解码器将尝试从 z 中恢复 $\tilde{x}$ 形式的输入。

从视觉上看，每个 MNIST 数字 x 看起来都类似于 $\tilde{x}$. 图 3.1.2 向我们展示了这个自动编码过程。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtcvms57fmj61ca0cmdhc02.jpg)

我们可以观察到解码后的数字 7 虽然不完全相同，但仍然足够接近。 由于编码器和解码器都是非线性函数，我们可以使用神经网络来实现两者。 例如，在 MNIST 数据集中，自动编码器可以通过 MLP 或 CNN 实现。 可以通过反向传播最小化损失函数来训练自编码器。 与其他神经网络类似，反向传播的一个要求是损失函数必须是可微的。

如果我们将输入视为分布，我们可以将编码器解释为分布的编码器 $p(z|x)$，将解码器解释为分布的解码器 $p(x|z)$。 自编码器的损失函数表示如下：

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtcvoo0csnj61a00440sr02.jpg)

损失函数简单意味着我们希望在考虑到潜在的矢量分布的情况下恢复输入分布的机会。 如果假定解码器输出分布为高斯，则损失函数会降至MSE：

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtd4u8f2ipj61bw07eaap02.jpg)

在该示例中，$𝒩（x_i;\tilde{x}_i，\sigma^2）$表示具有△的平均值的高斯分布和$\sigma^2$的方差。 假设恒定方差。 假设解码器输出$\tilde{x}_i$是独立的。 $m$是输出维度。

了解AutoEncoders后面的原则将帮助我们在代码实现中。 在下一节中，我们将查看如何使用TF.Keras功能API构建编码器，解码器和AutoEncoder。

## 使用Keras构建AutoEncoder

我们现在将转向真正令人兴奋的东西，使用TF.keras库构建AutoEncoder。 为简单起见，我们将使用Mnist DataSet进行第一组示例。 然后，AutoEncoder将从输入数据生成潜在的向量，并使用解码器恢复输入。 该第一示例中的潜伏载体是16℃。

首先，我们将通过构建编码器来实现AutoEncoder。

清单3.2.1显示了将Mnist数字压缩为16 dim潜在矢量的编码器。 编码器是两个CONV2D的堆栈。 最终阶段是具有16个单位的致密层以产生潜伏载体。

> 清单 3.2.1：autoencoder-mnist-3.2.1.py